In [1]:
import tensorflow as tf
import keras

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from IPython.display import display, HTML
import re
sensor_folder = Path('.') / 'Original_Data' / 'Sensor Data'
survey_file = Path('.') / 'Original_Data' / 'Survey' / 'Filmtrailer onderzoek juni-1 - MERGED_SURVEY_RESPONSE_MATRIX.txt'

In [3]:
def get_survey_data(file_path):
    survey_data = pd.read_csv(file_path.absolute(), sep = "\t", skiprows=57)
    survey_data = survey_data.drop(list(survey_data.filter(regex=("^LABELVALUE|LABELVALUE_Cijfer|TIMING"))), axis=1)
    
    d = {}

    for c in survey_data.columns[5:]:
        splitted = c.split(",")[0]
        trailer_number = re.findall(r"\d+", splitted)[0]
        if "LABELID_Film" in c:
            d[c] = "Seen_movie_" + str(trailer_number)
        elif "LABELID_Cijfer" in c:
            d[c] = "Rating_trailer_" + str(trailer_number)
        elif "VALUE Voorspelling" in c:
            d[c] = "Go_watch_Stimulus_" + str(trailer_number)
            
    survey_data = survey_data.rename(columns=d)
    
    return survey_data

survey_data = get_survey_data(survey_file)


display(HTML(survey_data.to_html()))

,STUDY,RESPONDENT,GENDER,AGE,GROUP,Seen_movie_3,Rating_trailer_3,Go_watch_Stimulus_3,Seen_movie_7,Rating_trailer_7,Go_watch_Stimulus_7,Seen_movie_2,Rating_trailer_2,Go_watch_Stimulus_2,Seen_movie_15,Rating_trailer_15,Go_watch_Stimulus_15,Seen_movie_9,Rating_trailer_9,Go_watch_Stimulus_9,Seen_movie_16,Rating_trailer_16,Go_watch_Stimulus_16,Seen_movie_1,Rating_trailer_1,Go_watch_Stimulus_1,Seen_movie_6,Rating_trailer_6,Go_watch_Stimulus_6,Seen_movie_10,Rating_trailer_10,Go_watch_Stimulus_10,Seen_movie_14,Rating_trailer_14,Go_watch_Stimulus_14,Seen_movie_12,Rating_trailer_12,Go_watch_Stimulus_12,Seen_movie_8,Rating_trailer_8,Go_watch_Stimulus_8,Seen_movie_5,Rating_trailer_5,Go_watch_Stimulus_5,Seen_movie_4,Rating_trailer_4,Go_watch_Stimulus_4,Seen_movie_13,Rating_trailer_13,Go_watch_Stimulus_13,Seen_movie_11,Rating_trailer_11,Go_watch_Stimulus_11,Unnamed: 133
0,Filmtrailer onderzoek juni,RESPONDENT 1,MALE,40,Default,Ja,9.0,4.0,Nee,5.0,2.0,Ja,7.0,3.0,Nee,1.0,1.0,Ja,5.0,2.0,Ja,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Filmtrailer onderzoek juni,RESPONDENT 6,MALE,46,Default,Nee,8.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,8.0,1.0,Nee,6.0,2.0,Nee,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,7.0,3.0,NaN,NaN,NaN,Nee,8.0,4.0,NaN
2,Filmtrailer onderzoek juni,RESPONDENT 7,MALE,45,Default,Ja,7.0,4.0,Nee,8.0,5.0,Nee,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nee,7.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,Nee,7.0,5.0,Nee,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Filmtrailer onderzoek juni,RESPONDENT 10,MALE,42,Default,Nee,7.0,3.0,NaN,NaN,NaN,Nee,5.0,2.0,NaN,NaN,NaN,Nee,5.0,2.0,NaN,NaN,NaN,Nee,4.0,2.0,NaN,NaN,NaN,Nee,5.0,2.0,Nee,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Filmtrailer onderzoek juni,RESPONDENT 12,MALE,71,Default,Nee,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nee,4.0,2.0,Nee,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nee,4.0,2.0,Nee,6.0,2.0,Nee,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Filmtrailer onderzoek juni,RESPONDENT 15,MALE,70,Default,Nee,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nee,8.0,4.0,Nee,6.0,3.0,Nee,8.0,4.0,NaN,NaN,NaN,Nee,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Nee,5.0,2.0,NaN,NaN,NaN,NaN
6,Filmtrailer onderzoek juni,RESPONDENT 17,MALE,45,Default,Nee,9.0,5.0,Ja,9.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,9.0,4.0,NaN,NaN,NaN,Nee,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,Nee,2.0,1.0,Nee,8.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Filmtrailer onderzoek juni,RESPONDENT 21,FEMALE,44,Default,Nee,9.0,5.0,Nee,9.0,5.0,Nee,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Nee,7.0,3.0,NaN,NaN,NaN,Nee,8.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nee,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Filmtrailer onderzoek juni,RESPONDENT 22,FEMALE,32,Default,Nee,8.0,4.0,Nee,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nee,7.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nee,8.0,4.0,Nee,6.0,2.0,NaN,NaN,NaN,Nee,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Filmtrailer onderzoek juni,RESPONDENT 26,FEMALE,20,Default,Nee,6.0,3.0,NaN,NaN,NaN,Ja,9.0,5.0,Nee,6.0,2.0,Nee,7.0,4.0,NaN,NaN,NaN,Nee,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,8.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
from mne_features.feature_extraction import extract_features


def extract_features_data(data, selected_features, sfreq=256):
    data_T = np.transpose(data, axes=[0,2,1])
    data_T = np.nan_to_num(data_T)
    extracted_features = extract_features(X = data_T, selected_funcs = selected_features, 
                                         sfreq = sfreq, return_as_df=True, n_jobs=-1)
    
    return extracted_features

# X_train_features = extract_features_data(data=X_train_filtered, selected_features= {'energy_freq_bands', 'katz_fd',
#                                                                                    'variance', 'std', 'ptp_amp', 'kurtosis',
#  'line_length',}
#                                                                                    )
# X_test_features = extract_features_data(data=X_test_filtered, selected_features= {'energy_freq_bands', 'katz_fd',
#                                                                                    'variance', 'std', 'ptp_amp', 'kurtosis',
#  'line_length',})

C:\Users\mleon\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [6]:
def get_respondent_data(file_path):
    sensor_data = pd.read_csv(file_path, skiprows=5, sep = "\t")
    sensor_data = sensor_data[['StudyName', 'Name', 'Age', 'Gender', 'StimulusName',
       'SlideType', 'Timestamp', 'POz (Raw)', 'Fz (Raw)', 'Cz (Raw)',
       'C3 (Raw)', 'C4 (Raw)', 'F3 (Raw)', 'F4 (Raw)', 'P3 (Raw)', 'P4 (Raw)']]
    
    survey_data = get_survey_data(survey_file)
    
    
    for rowval in sensor_data.StimulusName.unique():
        trailer_number = re.findall(r"\d+", rowval)[0]

        if "STIM" in rowval:
            sensor_data.replace(to_replace={"StimulusName":rowval}, value = "Stimulus_" + trailer_number, inplace=True)
        elif "Film al gezien" in rowval:
            sensor_data.replace(to_replace={"StimulusName":rowval}, value = "Question_Seen_Movie_" + trailer_number, inplace=True)

        elif "Cijfer" in rowval:
            sensor_data.replace(to_replace={"StimulusName":rowval}, value = "Question_Rate_Movie_" + trailer_number, inplace=True)

        elif "Voorspelling" in rowval:
            sensor_data.replace(to_replace={"StimulusName":rowval}, value = "Label_GoSee_Movie_" + trailer_number, inplace=True)
    
    
    participant_survey = survey_data.loc[survey_data.RESPONDENT == sensor_data.Name.unique()[0], :].dropna(axis=1)
    
    participant_labels = participant_survey.loc[:,[col for col in participant_survey if "Go" in col]]
    
    
    sensor_data = sensor_data.iloc[:,4:]
    unique_stimuli = sensor_data.StimulusName.unique()
    
    
    
    unique_stimuli = [s for s in unique_stimuli if "Stimulus" in s]
    
    sensor_data = sensor_data[sensor_data["StimulusName"].isin(unique_stimuli)]
    
    label_unique = participant_labels[["Go_watch_" + s for s in unique_stimuli if "Stimulus" in s]].values
    
    full_data = []

    columns_to_extract = sensor_data.columns[3:]
    
    for stim in unique_stimuli:
        
        if "Stimulus" in stim:
            arr = sensor_data.loc[sensor_data["StimulusName"] == stim, columns_to_extract].to_numpy()[0:65000,:]
            full_data.append(arr)
       
    full_data_arr = np.array(full_data)
    
    return full_data_arr, np.array(label_unique)



all_data = np.empty(shape=(32,6,65000,9))
all_labels = np.empty(shape=(32,6))

for i, file_path in enumerate(sensor_folder.iterdir()):
    
    sensor_data, label = get_respondent_data(file_path)    
    
    all_data[i] = sensor_data
    all_labels[i] = label

print(all_data.shape, all_labels.shape)

(32, 6, 65000, 9) (32, 6)


In [9]:
import mne

def filter_data(data, sfreq=256, l_freq=0.5, h_freq=35):
    data_t = np.transpose(data, axes=[0,2,1])
    filtered_t = mne.filter.filter_data(data = data_t, l_freq=l_freq, h_freq = h_freq, sfreq=sfreq)
    
    return np.transpose(filtered_t, axes=[0,2,1])

In [10]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier

def encode(all_labels):
    y = all_labels.copy()
    
    enc = LabelBinarizer()
    enc.fit(np.hstack(y.copy()))
    return enc

enc = encode(all_labels)

In [79]:
from keras.utils import to_categorical 


def new_rr(learning_rate = 0.0001, input_shape = (65000,9)):
    
    model = keras.models.Sequential()
    
    model.add(keras.layers.InputLayer(input_shape = input_shape))
    
    model.add(keras.layers.LSTM(20, return_sequences= True))
    
    model.add(keras.layers.LSTM(10, return_sequences=True))
    
    model.add(keras.layers.LSTM(5))
    
    model.add(keras.layers.Dense(6, activation="softmax"))
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics=["accuracy"])
    
    return model


In [81]:
m = new_rr()

m.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_23 (LSTM)               (None, 65000, 20)         2400      
_________________________________________________________________
lstm_24 (LSTM)               (None, 65000, 10)         1240      
_________________________________________________________________
lstm_25 (LSTM)               (None, 5)                 320       
_________________________________________________________________
dense_22 (Dense)             (None, 6)                 36        
Total params: 3,996
Trainable params: 3,996
Non-trainable params: 0
_________________________________________________________________


In [82]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [80]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier


model = KerasClassifier(new_rr,epochs=20, batch_size=30 )
for i in range(all_data.shape[0]):
    
    X_train = all_data.copy()
    y_train = all_labels.copy()
    
   
    
    X_test = X_train[i].copy()
    y_test = y_train[i].copy()
    
    X_train = np.delete(X_train, i, axis=0)
    y_train = np.delete(y_train, i, axis=0)
    
    
    
    X_train = np.vstack(X_train)
    
    X_train = filter_data(X_train)
    X_test = filter_data(X_test)

    
    y_test = to_categorical(y_test, num_classes = 6, dtype="int32")
    y_train = np.vstack(to_categorical(y_train, num_classes=6, dtype="int32"))
    
    X_train =  np.nan_to_num(X_train)
    X_test =  np.nan_to_num(X_test)

    model.fit(X_train, y_train)
    
    model.score(X_train, y_train)
    break




Setting up band-pass filter from 0.5 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 1691 samples (6.605 sec)

Setting up band-pass filter from 0.5 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)

ResourceExhaustedError:    OOM when allocating tensor with shape[1,30,5] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node gradients/CudnnRNN_grad/CudnnRNNBackprop}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[PartitionedCall]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_50050]

Function call stack:
train_function -> train_function -> train_function


In [ ]:
m = new_rr()
m.summary()

# Regular Classifiers with Feature Extraction

In [14]:
from mne_features.univariate import get_univariate_funcs
from keras.utils import to_categorical 

all_features = list(get_univariate_funcs(sfreq=256))

all_features

['app_entropy',
 'decorr_time',
 'energy_freq_bands',
 'higuchi_fd',
 'hjorth_complexity',
 'hjorth_complexity_spect',
 'hjorth_mobility',
 'hjorth_mobility_spect',
 'hurst_exp',
 'katz_fd',
 'kurtosis',
 'line_length',
 'mean',
 'pow_freq_bands',
 'ptp_amp',
 'samp_entropy',
 'skewness',
 'spect_edge_freq',
 'spect_entropy',
 'spect_slope',
 'std',
 'svd_entropy',
 'svd_fisher_info',
 'teager_kaiser_energy',
 'variance',
 'wavelet_coef_energy',
 'zero_crossings']

In [37]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=200, n_jobs=-1)

In [38]:
scores = []

for i in range(all_data.shape[0]):
    
    X_train = all_data.copy()
    y_train = all_labels.copy()
    
    X_test = X_train[i].copy()
    
    ##### filtered_data###############
    X_test_filtered = filter_data(X_test)
    ##################################
    y_test = y_train[i].copy()
    X_train = np.delete(X_train, i, axis=0)
    y_train = np.delete(y_train, i, axis=0)
    
    X_train = np.vstack(X_train)
    
    ######### filtered_data #################
    X_train_filtered = filter_data(X_train)
    #########################################
    
    ############### regular classifiers ############################
    X_train_features = extract_features_data(data=X_train_filtered, selected_features= {'energy_freq_bands'}
                                                                                   )
    X_test_features = extract_features_data(data=X_test_filtered, selected_features= {'energy_freq_bands'})
    
    
    
    
    
    #########################################################
    
    ######## encode labels ###################################
    y_test = to_categorical(y_test, num_classes=6)
    y_train = np.vstack(to_categorical(y_train, num_classes=6))
    
    y_train_clf = np.argmax(y_train, axis=1)
    y_test_clf = np.argmax(y_test, axis= 1)
    
    clf.fit(X_train_features, y_train_clf)
    
    scores.append(clf.score(X_test_features, y_test_clf))


Setting up band-pass filter from 0.5 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 1691 samples (6.605 sec)

Setting up band-pass filter from 0.5 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)

- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 1691 samples (6.605 sec)

Setting up band-pass filter from 0.5 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 1691 samples (6.605 sec)

Setting up band-pass filter from 0.5 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth

- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 1691 samples (6.605 sec)

Setting up band-pass filter from 0.5 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 1691 samples (6.605 sec)

Setting up band-pass filter from 0.5 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 pas

- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 1691 samples (6.605 sec)

Setting up band-pass filter from 0.5 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 1691 samples (6.605 sec)

Setting up band-pass filter from 0.5 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pa


Setting up band-pass filter from 0.5 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 1691 samples (6.605 sec)



0.2864583333333333